sarahfong
# Overview
This notebook details how to set up a *fairly* seamless wynton login/environment

- Here I detail:
    1. how to get setup on wynton 
    2. how to login to wynton *without using your password*
    3. how to set up a virtual environment with micromamba and jupyter notebooks on wynton
    4. how to tunnel from your local computer to wynton

# How to get setup on wynton

## Request a wynton account

website - https://wynton.ucsf.edu/hpc/about/join.html
    
1. select "Request a Wynton Account / Modify an Existing Account (requires UCSF MyAccess login)"
2. login to MyAccess
3. set Nadav as the PI in UCSF PI Email
4. Which email address do I use? 
    - UCSF email account, preferrable.
    - personal email address, less preferrable. 
5. once you receive your username and password from wynton, login. 

# Login to wynton

In **local terminal**, use your username, and enter


- replace "USERNAME" with your username



In [ ]:


ssh USERNAME@log1.wynton.ucsf.edu



## Login to wynton without password
- In the section below, I try to recap steps 1-4 outlined here: https://wynton.ucsf.edu/hpc/howto/log-in-without-pwd.html


- At bottom of page, follow "Connect directly to a development node" steps. 

**THIS WILL SAVE YOU SO MUCH TIME**

1. Setup public-private ssh key, passphrase
 
- generate private public ssh key, copy it to wynton
- Run in **local terminal**

### generate key
- In **local terminal** 

In [ ]:


mkdir ./.ssh  # make directory

chmod u=rwx,go= ./.ssh  # change permissions for directory 

ssh-keygen -m PEM -f ./.ssh/laptop_to_wynton  # generate the key



### copy key to wynton

- change USERNAME to your username
- Run in **local terminal**

In [ ]:


ssh-copy-id -i ./.ssh/laptop_to_wynton.pub USERNAME@log1.wynton.ucsf.edu  

# this command will prompt login to add key to wynton



### login to wynton using key
- change USERNAME to your username
- Run in **local terminal**

In [ ]:


ssh -o PreferredAuthentications=publickey,keyboard-interactive -o IdentitiesOnly=yes -i ./.ssh/laptop_to_wynton USERNAME@log1.wynton.ucsf.edu



### set up ssh config, add wynton info
- make config 
- Run in **local terminal**



In [ ]:


touch ./.ssh/config    



### add login info to config
- change USERNAME to your username
- Run in **local terminal**

In [ ]:


echo 'Host *.wynton.ucsf.edu' >>./.ssh/config

echo 'User USERNAME' >>./.ssh/config

echo 'IdentityFile ./.ssh/laptop_to_wynton' >>./.ssh/config



### add proxyjump to config to login directly to dev2
- Run in **local terminal**

In [ ]:


echo 'Host *dev?.wynton.ucsf.edu' >>./.ssh/config
echo 'ProxyJump log1.wynton.ucsf.edu' >>./.ssh/config



#### test proxy jump

In [ ]:


# open new local terminal

ssh dev2.wynton.ucsf.edu



# Set up mamba virtual environment and Jupyter Notebooks
- Important for running software packages into your environment without destroying wynton for everyone else. 




## download micromamba locally
from https://mamba.readthedocs.io/en/latest/installation/micromamba-installation.html

- Why micromamba?
    - Micromamba functions *almost exactly* like conda, but much faster because it can parallelize package downloads. 
    - Wynton does not support micromamba, so you have to download a copy to your home directory and set it up.
    - use "micromamba" like you would "conda"
        - e.g. micromamba install ..., micromamba activate...  


In [ ]:


# log on to dev2 gateway on wynton
ssh dev2.wynton.ucsf.edu

curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba 



### add shortcut to load micromamba upon login 

In [ ]:


# on dev2.wynton.ucsf.edu

./bin/micromamba shell init -s bash -p ~/micromamba  # this writes to your .bashrc file

# sourcing the bashrc file incorporates the changes into the running session.
# better yet, restart your terminal!

source ~/.bashrc



## create virtual environment

In [ ]:


# on dev2.wynton.ucsf.edu

micromamba create -n myenv



## activate your environment

In [ ]:


# on dev2.wynton.ucsf.edu

micromamba activate myenv



## download jupyter notebooks 

In [ ]:


# on dev2.wynton.ucsf.edu

micromamba install -c conda-forge jupyter  # command to install jupyter from conda-forge channel (-c)



# Setup tunneling wynton
- copy launch script to home directory, rename the template file to "launch_nb.sh"

In [ ]:


# on dev2.wynton.ucsf.edu

cp /wynton/group/ahituv/bin/template_launch_nb.sh $HOME && mv $HOME/template_launch_nb.sh $HOME/launch_nb.sh



## Get a PORTNUMBER

In [ ]:


# on dev2.wynton.ucsf.edu

module load CBI port4me

port4me --tool=jupyter

# e.g. this made 16171



- write down this port number

## write unique port number into launch script
- replace **NEW_PORT_NUMBER** with the portnumber generated from port4me (above)
    - sed command searches and replaces PORT=00000 with your portnumber, writes to the file 't'
    - then, we rename 't' as the new launch_nb.sh file

In [ ]:


# on dev2.wynton.ucsf.edu

sed 's/PORT=00000/PORT=**NEW_PORT_NUMBER**/' $HOME/launch_nb.sh > t && mv t  $HOME/launch_nb.sh



# Tunneling to wynton

## After setup, use this everytime you want to tunnel to wynton

- step 1: logon wynton
- step 2: launch jupyter notebook job with qsub command
- step 3: get tunnel ssh command
- step 4: in local terminal, tunnel to job.
- step 5: start session in local browser

## Step 1: login to wynton

In [ ]:


# on local terminal
ssh dev2.wynton.ucsf.edu



## Step 2: Submit job
- step 2.1: submit job
- step 2.2: check on job state to see that it is running
- step 2.3: set jobnumber bash variable

In [ ]:


# step 2.1: on dev2.wynton.ucsf.edu

qsub launch_nb.sh

# Your job 9994008 ("launch_nb.sh") has been submitted
# remember the qsub job number (e.g. 9994008). You will need this for the next step!



In [ ]:


# step 2.2: check job status

qstat

## when state is 'r' the job is running
## when state is "qw" wait a little longer for state to run, 'r'



In [ ]:


# step 2.3 set job number bash variable (e.g.  qsub job number was 9994008)

jobnumber=9994008  

# CHANGE 9994008 to your job number!



## Step 3: get tunnel ssh from the session_output.txt

- head the job session

In [ ]:


head $jobnumber_jupyter_session.output.txt



- read head, copy the command that starts with **"ssh -L 9999:msg-id6.wynton.ucsf.edu:16171 USERNAME@log1.wynton.ucsf.edu "**


In [ ]:
""" e.g.
To open a tunnel from local machine,
Execute in a new terminal window:
ssh -L 9999:msg-id6.wynton.ucsf.edu:16171 fongsl@log1.wynton.ucsf.edu   ### COPY THE LINE THAT LOOKS LIKE THIS ###
================================================================================
"""

## Step 4: tunnel to wynton session
- Step 4.1: Open up new terminal window on local computer
- Step 4.2: paste ssh -L command (from step 3), login to wynton. 


In [ ]:


# step 4.1: open a new local terminal (NOT WYNTON)

# step 4.2: run the command from the head step above

ssh -L 9999:msg-id6.wynton.ucsf.edu:16171 fongsl@log1.wynton.ucsf.edu



## Step 5: open web browser
- can be chrome, safari, firefox, internet explorer, etc. 
- go to web address: http://localhost:9999

Remember **both your password and passphrase**

And you did it!

# Extra: Jupyter notebook kernels
- when you want to run jupyter notebook in a specific environment. 

In [ ]:


# login to wynton
ssh dev2.wynton.ucsf.edu

# activate your environment
micromamba active myenv

# add your environment as an ipython kernel
python -m ipykernel install --user --name myenv --display-name "myenv"



- tunnel to jupyter notebooks 
- open a new notebook
- under the 'kernel' tab select "change kernel" and select "myenv"

that's it. You can now run Jupyter notebooks using your brand new virtual environment!